# Segmentation 3D

## 1. Prerequisites

### 1.1 Load cloud point of voxel centers

In [1]:
import cv2

import openalea.phenomenal.display as display
import openalea.phenomenal.data as data

from openalea.phenomenal.display.notebook import *

from openalea.phenomenal.object import (
    VoxelSkeleton,
    VoxelGrid,
    VoxelSegmentation,
    ImageView)

from openalea.phenomenal.multi_view_reconstruction import *

from openalea.phenomenal.segmentation_3D import (
    voxel_graph_from_voxel_grid,
    skeletonize,
    segment_reduction,
    maize_segmentation,
    maize_analysis)

## Display voxel grid

In [2]:
plant_number = 11
voxels_size = 16
bin_images = data.bin_images(plant_number=plant_number)
calibrations = data.calibrations(plant_number=plant_number)
voxel_grid = data.voxel_grid(plant_number=plant_number, voxels_size=voxels_size)

show_voxel_grid(voxel_grid)

A Jupyter Widget

## Skeletonization

Compute the skeleton of the plant voxelgrid

In [3]:
import time
t0 = time.time()

subgraph = None
voxel_graph = voxel_graph_from_voxel_grid(voxel_grid, connect_all_point=True)
voxel_skeleton = skeletonize(voxel_graph.graph,
                             voxel_graph.voxels_size,
                             subgraph=subgraph)

### Display it

In [4]:
show_skeleton(voxel_skeleton)

A Jupyter Widget

# Voxel segment reduction

In [5]:
# Select images
image_views = list()
for angle in bin_images["side"]:
    projection = calibrations["side"].get_projection(angle)
    image_views.append(ImageView(bin_images["side"][angle], projection))

voxel_skeleton_reduced = segment_reduction(voxel_skeleton,
                                           image_views,
                                           tolerance=4,
                                           nb_min_pixel=100)

### Display it

In [6]:
show_skeleton(voxel_skeleton_reduced)

A Jupyter Widget

# Segmentation Maize

In [11]:
vms = maize_segmentation(voxel_skeleton_reduced, voxel_graph)

import openalea.phenomenal.display as display

ds = display.DisplaySegmentation()
ds(vms)

filename = 'voxel_maize_segmentation.json'
vms.write_to_json_gz(filename)
vms = VoxelSegmentation.read_from_json_gz(filename)

In [8]:
show_segmentation(vms)

A Jupyter Widget

In [9]:
vmsi = maize_analysis(vms)
file_prefix = 'voxel_maize_segmentation_info'
vmsi.write_to_json_gz(file_prefix)
vmsi = VoxelSegmentation.read_from_json_gz(file_prefix)

In [10]:
show_segmentation(vmsi, size_ratio=2)

A Jupyter Widget